# Types 

The **type** of a variable tells us the "shape" of the variable, i.e. how to treat / interpret the data stored in the block of memory associated with that variable.

Although it is possible to do much in Julia without thinking or worrying about types, they are always just under the surface, and the true power of the language becomes available through their use.

In [ ]:
x = 3

In [ ]:
typeof(x)

For basic ("primitive") types, we can see the raw bits that are associated to a variable:

In [ ]:
bits(x)  # bitstring in Julia 0.7

In [ ]:
y = 3.0

In [ ]:
bits(y)

The internal representations are different, corresponding to the different types. 

We can treat the storage as being of a different type:

In [ ]:
z = reinterpret(Int, y)

In [ ]:
h = hex(z)

In [ ]:
z1 = parse(Int, h, 16)  # base 16

In [ ]:
bits(z1)

# Defining our own types

We have previously seen some examples of how to define a new type in Julia. 

A type definition can be thought of as template for a kind (type) of box, that contains certain **fields** containing data.

One of the simplest examples is a "volume" type, representing the volume of some physical or mathematical object:

In [ ]:
struct Vol
    value
end

In [ ]:
Vol

This defines a new type, called `Vol`, containing one field, called `value`. 

It does not yet create an object that has that type. That is done by calling a **constructor** -- a special function with the same name as the type:

In [ ]:
V = Vol(3)

In [ ]:
V

`V` is a Julia variable that is of type `Vol`:

In [ ]:
typeof(V)

Its "shape" is thus that of a box containing itself a single variable, which we can access as

In [ ]:
V.value

Since we defined the type `Vol` as `struct`, we cannot modify the contents of the object once it has been created:

In [ ]:
V.value = 10

We could instead make a mutable object:

In [ ]:
mutable struct Vol1
    value
end

In [ ]:
V = Vol1(3)

In [ ]:
V.value

In [ ]:
V.value = 10

In [ ]:
V

We can change how our objects look by defining a new method of the `show` function:

In [ ]:
import Base: show

show(io::IO, V::Vol1) = print(io, "Volume with value ", V.value)

In [ ]:
V = Vol1(3)

We can define e.g. the sum of two volumes:

In [ ]:
import Base: +

+(V1::Vol1, V2::Vol1) = Vol1(V1.value + V2.value)

In [ ]:
V + V

But the following does not work, since we haven't defined `*` yet on our type:

In [ ]:
2V

**Exercise**: Define `*` of two `Vol`s  and of a `Vol` and a number.

## Type annotations

There is a problem with our definition:

In [ ]:
Vol1("hello")

It doesn't make sense to have a string as a volume. So we should **restrict** which kinds of `value` are allowed, by specifying ("annotating") the type of `value` in the type definition:

In [ ]:
struct Vol2
    value::Float64
end

In [ ]:
Vol2(3.1)

In [ ]:
Vol2("hello")

# Parameterizing a type

In different contexts, we may want integer volumes, or rational volumes, rather than `Vol`s which contain a floating-point number, e.g. for a 3D printer that makes everything out of cubes of the same size. 

We could define the following sequence of different types.

In [ ]:
type Vol_Int
    value::Int
end

type Vol_Float
    value::Float64
end

type Vol_Rational
    value::Rational{Int64}
end

In [ ]:
Vol_Int(3)

In [ ]:
Vol_Int(3.1)

In [ ]:
Vol_Float(3.1)

But clearly this is the wrong way to do it, since we're repeating ourselves, leading to inefficiency and buggy code. (https://en.wikipedia.org/wiki/Don't_repeat_yourself).

Can't Julia itself automatically generate all of these different types?

## Specifying type parameters

What we would like to do is tell Julia that the **type** itself (here, `Int`, `Float64` or `Rational{Int64}`) 
is a special kind of **parameter** that we will specify. 

To do so, we use curly braces (`{`, `}`) to specify such **type parameter** `T`:

In [ ]:
struct Vol3{T}
    value
end

We can now create objects of type `Vol3`  with any type `T`:

In [ ]:
V = Vol3{Float64}(3.1)

In [ ]:
typeof(V)

In [ ]:
V2 = Vol3{Int64}(4)

In [ ]:
typeof(V2)

We see that the types of `V1` and `V2` are *different* (but related), and we have achieved what 
we wanted.

The type `Vol3` is called a **parametric type**, with **type parameter** `T`. Parameteric types may have several type parameters, as we have already seen with `Array`s:

In [ ]:
a = [3, 4, 5]
typeof(a)

The type parameters here are `Int64`, which is itself a type, and the number `1`.

## Improving the solution

The problem with this solution is the following, which echos what happened at the start of the notebook:

In [ ]:
V = Vol3{Int64}(3.1)

In [ ]:
typeof(V.value)

The type `Float64` of the field `V.value` is distinct from the type parameter `Int64` we specified. 
So we have not yet actually captured the pattern of `Vol2`,
which restricted the `value` field to be of the desired type.

We solve this be specifying the field to **also be of type `T`**, with the **same `T`**:

In [ ]:
struct Vol4{T}
    value::T
end

For example,

In [ ]:
V = Vol4{Int64}(3)

If necessary and possible, the argument to the constructor will be converted to the parametric type `T` that we specify:

In [ ]:
V = Vol4{Int64}(3.0)

In [ ]:
typeof(V.value)

Now when we try to do 

In [ ]:
Vol4{Int64}(3.1)

Julia throws an error, namely an `InexactError()`.
This means that we are trying to "force" the number 3.1 into a "smaller" type `Int64`, i.e. one in which it can't be represented.

However, now we seem to be repeating ourselves again: We know that `3.1` is of type `Float64`, and in fact Julia knows this too; so it seems redundant to have to specify it. Can't Julia just work it out? Indeed, it can!:

In [ ]:
Vol4(3.1)

Here, Julia has **inferred** the type `T` from the "inside out". That is, it performed pattern matching to realise that `value::T` is **matched** if `T` is chosen to be `Float64`, and then propagated this same value of `T` **upwards** to the type parameter.

## More fields

**Exercise**: Define a `Point` type that represents a point in 2D, with two fields. What are the options for this type, mirroring the types `Vol1` through `Vol4`?

## Summary

With parametric types, we have the following possibilities:

1. Julia converts (if possible) to the header type

2. Julia infers the header type from the inside (through the argument)


# Constructors

When we define a type, Julia also defines the **constructor functions** that we have been using above. These are functions with exactly the same name as the type.

They can be discovered using `methods`:

In [ ]:
struct Vol1
    value
end

In [ ]:
methods(Vol1)

We see that Julia provides two default constructors. [Note that the output has changed in Julia 0.7.]

For parametric types, it is a bit more complicated:

In [ ]:
methods(Vol4)

In [ ]:
methods(Vol4{Float64})

## Outer constructors

Julia allows us to provide our own constructor functions.
E.g.

In [ ]:
struct Vol1
    value
end

In [ ]:
struct Vol2
    value::Float64
end

In [ ]:
Vol2(3)

In [ ]:
Vol2("3.1")

Here, we have tried to provide a numeric string, which is not allowed, since the string is not a number. We can add a constructor to allow this:

In [ ]:
Vol2(s::String) = Vol2(parse(Float64, s))

In [ ]:
Vol2("3.1")

In [ ]:
Vol2("hello")

We have added a new constructor outside the type definition, so it is called an **outer constructor**.

## Constructors that impose a restriction: **inner constructors**

Now consider the following:

In [ ]:
Vol4(3)

In [ ]:
V = Vol4(3.5)

In [ ]:
V.value

In [ ]:
Vol4(-1)

Oops! A volume cannot be negative, but the constructors so far have no restrictions, and so allow us to make a negative volume. To prevent this, Julia allows us to provide our own constructor, in which any restrictions are enforced.

These constructors are written **within the type definition itself**, and hence are called **inner constructors**.

[In Julia, these are the **only methods** that may be defined inside the type definition. Unlike in object-oriented languages, methods **do not belong to types** in Julia; rather, they exist outside any particular type, and (multiple) dispatch is used instead.]

For example:

In [ ]:
struct Vol5
    value::Float64
    
    function Vol5(V) 
        if V < 0
            throw(ArgumentError("Volumes cannot be negative"))
        end
        
        new(V)
    end
end

In [ ]:
Vol5(3)

In [ ]:
Vol5(-34)

If we define an inner constructor, then Julia no longer defines the standard constructors; this is why defining an inner constructor gives us exclusive control over how our objects are created.

Note that we use a special function `new` to actually create the object by filling in the values of the fields.

If we use an immutable object (defined using `struct`), there is no way of changing the value of the field stored inside the object, so the invariant that `value` must be positive can never be violated.

# Parametric functions

Since we now have the ability to make parametric types, we may wish to define parametric functions on those types. E.g.

In [ ]:
struct Length{T}
    length::T
end

In [ ]:
l = Length(10)

In [ ]:
function square_area(l)
    return l^2
end

Suppose that we wish to round the area of squares with floating-point side length:

In [ ]:
function square_area(l::T) where {T <: AbstractFloat}  # method for types T that are subtypes of AbstractFloat
    return ceil(Int, l^2)
end

In [ ]:
square_area(11.1)

# Inner constructors for parametric types

In [ ]:
struct Vol6{T<:Real}
    value::T
    
    function Vol6{T}(V) where {T<:Real}   # where specifies that T is a parameter of the parametric function Vol6
        if V < 0
            throw(ArgumentError("Negative"))
        end

        return new{T}(V)
    end
end

Here, we have used the syntax for parametric functions to specify a parametric inner constructor

In [ ]:
Vol6(3)

In [ ]:
Vol6{Float64}(3.1)

We see that so far, we must explicitly specify the parametric type.

In [ ]:
methods(Vol6)

We can again make Julia infer the type for us:

In [ ]:
Vol6(x::T) where {T<:Real} = Vol6{T}(x)

In [ ]:
Vol6(3.1)

In [ ]:
Vol6(3)

In [ ]:
methods(Vol6)

In [ ]:
x = 3//4  # rational number

In [ ]:
Vol6(x)

# Fixed-size objects are efficient

In [ ]:
struct Vec{T}
    x::T
    y::T
end

How efficient is this?

In [ ]:
import Base: +

+(f::Vec{T}, g::Vec{T}) where {T} = Vec(f.x + g.x, f.y + g.y)

In [ ]:
using BenchmarkTools

In [ ]:
@btime +(Vec(1.0, 2.0), Vec(1.0, 2.0))

In [ ]:
@btime [1.0, 2.0] + [1.0, 2.0]

Using fixed-size objects is much more efficient (50 times more efficient)! They are defined in the `StaticArrays.jl` package.